In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

# Import our input dataset
loans_df = pd.read_csv('./Data/loan_status.csv')
loans_df.head()

,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
0,Fully_Paid,99999999,Short_Term,741.0,2231892.0,8_years,Own_Home,Debt_Consolidation,29200.53,14.9,29.0,18,1,297996,750090.0,0.0,0.0
1,Fully_Paid,217646,Short_Term,730.0,1184194.0,<_1_year,Home_Mortgage,Debt_Consolidation,10855.08,19.6,10.0,13,1,122170,272052.0,1.0,0.0
2,Fully_Paid,548746,Short_Term,678.0,2559110.0,2_years,Rent,Debt_Consolidation,18660.28,22.6,33.0,4,0,437171,555038.0,0.0,0.0
3,Fully_Paid,99999999,Short_Term,728.0,714628.0,3_years,Rent,Debt_Consolidation,11851.06,16.0,76.0,16,0,203965,289784.0,0.0,0.0
4,Fully_Paid,99999999,Short_Term,740.0,776188.0,<_1_year,Own_Home,Debt_Consolidation,11578.22,8.5,25.0,6,0,134083,220220.0,0.0,0.0


In [2]:
# Generate our categorical variable list
loans_cat = loans_df.dtypes[loans_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
loans_df[loans_cat].nunique()

Loan_Status              2
Term                     2
Years_in_current_job    11
Home_Ownership           4
Purpose                  7
dtype: int64

In [3]:
# Check the unique value counts to see if binning is required
loans_df.Years_in_current_job.value_counts()

10+_years    13149
2_years       3225
3_years       2997
<_1_year      2699
5_years       2487
4_years       2286
1_year        2247
6_years       2109
7_years       2082
8_years       1675
9_years       1467
Name: Years_in_current_job, dtype: int64

In [5]:
# Create a OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(loans_df[loans_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(loans_cat)
encode_df.head()

,Loan_Status_Fully_Paid,Loan_Status_Not_Paid,Term_Long_Term,Term_Short_Term,Years_in_current_job_10+_years,Years_in_current_job_1_year,Years_in_current_job_2_years,Years_in_current_job_3_years,Years_in_current_job_4_years,Years_in_current_job_5_years,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
# Merge one-hot encoded features and drop the originals
loans_df = loans_df.merge(encode_df,left_index=True, right_index=True)
loans_df = loans_df.drop(loans_cat,1)
loans_df.head()

,Current_Loan_Amount,Credit_Score,Annual_Income,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,99999999,741.0,2231892.0,29200.53,14.9,29.0,18,1,297996,750090.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,217646,730.0,1184194.0,10855.08,19.6,10.0,13,1,122170,272052.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,548746,678.0,2559110.0,18660.28,22.6,33.0,4,0,437171,555038.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,99999999,728.0,714628.0,11851.06,16.0,76.0,16,0,203965,289784.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,99999999,740.0,776188.0,11578.22,8.5,25.0,6,0,134083,220220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [7]:
# Remove loan status target from features data
y = loans_df.Loan_Status_Fully_Paid
X = loans_df.drop(columns=["Loan_Status_Fully_Paid","Loan_Status_Not_Paid"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.849


In [9]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 27317 samples
Epoch 1/50
27317/27317 [==============================] - 2s 58us/sample - loss: 0.4205 - accuracy: 0.8401
Epoch 2/50
27317/27317 [==============================] - 1s 41us/sample - loss: 0.3862 - accuracy: 0.8496
Epoch 3/50
27317/27317 [==============================] - 1s 39us/sample - loss: 0.3817 - accuracy: 0.8490
Epoch 4/50
27317/27317 [==============================] - 1s 37us/sample - loss: 0.3798 - accuracy: 0.8493
Epoch 5/50
27317/27317 [==============================] - 1s 50us/sample - loss: 0.3785 - accuracy: 0.8493
Epoch 6/50
27317/27317 [==============================] - 1s 52us/sample - loss: 0.3776 - accuracy: 0.8494
Epoch 7/50
27317/27317 [==============================] - 1s 46us/sample - loss: 0.3768 - accuracy: 0.8495
Epoch 8/50
27317/27317 [==============================] - 1s 39us/sample - loss: 0.3760 - accuracy: 0.8493s -
Epoch 9/50
27317/27317 [==============================] - 1s 41us/sample - loss: 0.3756 - accuracy: 0.8498
Epoch 10/50

In [10]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 27317 samples
Epoch 1/100
   32/27317 [..............................] - ETA: 6:50 - loss: 0.3005 - accuracy: 0.9062
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 1024/27317 [>.............................] - ETA: 22s - loss: 0.3610 - accuracy: 0.8496 
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 2048/27317 [=>............................] - ETA: 12s - loss: 0.3659 - accuracy: 0.8457
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 3072/27317 [==>...........................] - ETA: 8s - loss: 0.3667 - accuracy: 0.8444 
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 4096/27317 [===>..........................] - ETA: 6s - loss: 0.3681 - accuracy: 0.8457
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 5120/27317 [====>.........................] - ETA: 5s - loss: 0.3646 - accuracy: 0.8469
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 6144/27317 [=====>........................] - ETA: 4s - loss: 0.3604 - accuracy: 0

  960/27317 [>.............................] - ETA: 1s - loss: 0.3508 - accuracy: 0.8594
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 1792/27317 [>.............................] - ETA: 1s - loss: 0.3687 - accuracy: 0.8477
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 2688/27317 [=>............................] - ETA: 1s - loss: 0.3607 - accuracy: 0.8519
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 3712/27317 [===>..........................] - ETA: 1s - loss: 0.3508 - accuracy: 0.8572
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 4736/27317 [====>.........................] - ETA: 1s - loss: 0.3508 - accuracy: 0.8581
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 5760/27317 [=====>........................] - ETA: 1s - loss: 0.3551 - accuracy: 0.8559
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 6784/27317 [======>.......................] - ETA: 1s - loss: 0.3561 - accuracy: 0.8560
Epoch 00003: saving model to checkp

 2400/27317 [=>............................] - ETA: 1s - loss: 0.3683 - accuracy: 0.8483
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 3328/27317 [==>...........................] - ETA: 1s - loss: 0.3601 - accuracy: 0.8531
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 4352/27317 [===>..........................] - ETA: 1s - loss: 0.3602 - accuracy: 0.8527
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 5376/27317 [====>.........................] - ETA: 1s - loss: 0.3552 - accuracy: 0.8558
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 7392/27317 [=======>......................] - ETA: 1s - loss: 0.3618 - accuracy: 0.8515
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 7424/27317 [=======>......................] - ETA: 1s - loss: 0.3614 - accuracy: 0.8518
Epoch 00005: saving model to checkpoints/weights.05.hdf5
 8448/27317 [========>.....................] - ETA: 1s - loss: 0.3646 - accuracy: 0.8496
Epoch 00005: saving model to checkp

 3968/27317 [===>..........................] - ETA: 1s - loss: 0.3555 - accuracy: 0.8556
Epoch 00007: saving model to checkpoints/weights.07.hdf5
 4992/27317 [====>.........................] - ETA: 1s - loss: 0.3565 - accuracy: 0.8530
Epoch 00007: saving model to checkpoints/weights.07.hdf5
 6016/27317 [=====>........................] - ETA: 1s - loss: 0.3576 - accuracy: 0.8526
Epoch 00007: saving model to checkpoints/weights.07.hdf5
 7040/27317 [======>.......................] - ETA: 1s - loss: 0.3557 - accuracy: 0.8543
Epoch 00007: saving model to checkpoints/weights.07.hdf5
 8064/27317 [=======>......................] - ETA: 1s - loss: 0.3542 - accuracy: 0.8553
Epoch 00007: saving model to checkpoints/weights.07.hdf5
 9088/27317 [========>.....................] - ETA: 1s - loss: 0.3533 - accuracy: 0.8554
Epoch 00007: saving model to checkpoints/weights.07.hdf5
10112/27317 [==========>...................] - ETA: 1s - loss: 0.3530 - accuracy: 0.8560
Epoch 00007: saving model to checkp

 5632/27317 [=====>........................] - ETA: 1s - loss: 0.3658 - accuracy: 0.8484
Epoch 00009: saving model to checkpoints/weights.09.hdf5
 6656/27317 [======>.......................] - ETA: 1s - loss: 0.3605 - accuracy: 0.8520
Epoch 00009: saving model to checkpoints/weights.09.hdf5
 7680/27317 [=======>......................] - ETA: 1s - loss: 0.3605 - accuracy: 0.8516
Epoch 00009: saving model to checkpoints/weights.09.hdf5
 8704/27317 [========>.....................] - ETA: 1s - loss: 0.3607 - accuracy: 0.8524
Epoch 00009: saving model to checkpoints/weights.09.hdf5
 9728/27317 [=========>....................] - ETA: 1s - loss: 0.3609 - accuracy: 0.8528
Epoch 00009: saving model to checkpoints/weights.09.hdf5
10752/27317 [==========>...................] - ETA: 1s - loss: 0.3594 - accuracy: 0.8537
Epoch 00009: saving model to checkpoints/weights.09.hdf5
11776/27317 [===========>..................] - ETA: 0s - loss: 0.3588 - accuracy: 0.8541
Epoch 00009: saving model to checkp

...

   32/27317 [..............................] - ETA: 1s - loss: 0.3214 - accuracy: 0.8750
Epoch 00038: saving model to checkpoints/weights.38.hdf5
 1088/27317 [>.............................] - ETA: 1s - loss: 0.3318 - accuracy: 0.8631
Epoch 00038: saving model to checkpoints/weights.38.hdf5
 1728/27317 [>.............................] - ETA: 1s - loss: 0.3361 - accuracy: 0.8611
Epoch 00038: saving model to checkpoints/weights.38.hdf5
 2624/27317 [=>............................] - ETA: 1s - loss: 0.3340 - accuracy: 0.8659
Epoch 00038: saving model to checkpoints/weights.38.hdf5
 3648/27317 [===>..........................] - ETA: 1s - loss: 0.3393 - accuracy: 0.8618
Epoch 00038: saving model to checkpoints/weights.38.hdf5
 4672/27317 [====>.........................] - ETA: 1s - loss: 0.3475 - accuracy: 0.8562
Epoch 00038: saving model to checkpoints/weights.38.hdf5
 5696/27317 [=====>........................] - ETA: 1s - loss: 0.3505 - accuracy: 0.8543
Epoch 00038: saving model to checkp

 2080/27317 [=>............................] - ETA: 1s - loss: 0.3685 - accuracy: 0.8413
Epoch 00040: saving model to checkpoints/weights.40.hdf5
 2944/27317 [==>...........................] - ETA: 1s - loss: 0.3702 - accuracy: 0.8427
Epoch 00040: saving model to checkpoints/weights.40.hdf5
 3776/27317 [===>..........................] - ETA: 1s - loss: 0.3637 - accuracy: 0.8464
Epoch 00040: saving model to checkpoints/weights.40.hdf5
 4352/27317 [===>..........................] - ETA: 1s - loss: 0.3644 - accuracy: 0.8456
Epoch 00040: saving model to checkpoints/weights.40.hdf5
 5312/27317 [====>.........................] - ETA: 1s - loss: 0.3633 - accuracy: 0.8473
Epoch 00040: saving model to checkpoints/weights.40.hdf5
 6336/27317 [=====>........................] - ETA: 1s - loss: 0.3561 - accuracy: 0.8494
Epoch 00040: saving model to checkpoints/weights.40.hdf5
 7360/27317 [=======>......................] - ETA: 1s - loss: 0.3564 - accuracy: 0.8500
Epoch 00040: saving model to checkp

 2880/27317 [==>...........................] - ETA: 1s - loss: 0.3635 - accuracy: 0.8451
Epoch 00042: saving model to checkpoints/weights.42.hdf5
 3904/27317 [===>..........................] - ETA: 1s - loss: 0.3519 - accuracy: 0.8530
Epoch 00042: saving model to checkpoints/weights.42.hdf5
 4928/27317 [====>.........................] - ETA: 1s - loss: 0.3549 - accuracy: 0.8504
Epoch 00042: saving model to checkpoints/weights.42.hdf5
 5952/27317 [=====>........................] - ETA: 1s - loss: 0.3543 - accuracy: 0.8520
Epoch 00042: saving model to checkpoints/weights.42.hdf5
 6976/27317 [======>.......................] - ETA: 1s - loss: 0.3558 - accuracy: 0.8519
Epoch 00042: saving model to checkpoints/weights.42.hdf5
 8000/27317 [=======>......................] - ETA: 1s - loss: 0.3571 - accuracy: 0.8510
Epoch 00042: saving model to checkpoints/weights.42.hdf5
 9024/27317 [========>.....................] - ETA: 1s - loss: 0.3579 - accuracy: 0.8493
Epoch 00042: saving model to checkp

 5536/27317 [=====>........................] - ETA: 1s - loss: 0.3528 - accuracy: 0.8492
Epoch 00044: saving model to checkpoints/weights.44.hdf5
 5568/27317 [=====>........................] - ETA: 1s - loss: 0.3542 - accuracy: 0.8482
Epoch 00044: saving model to checkpoints/weights.44.hdf5
 6592/27317 [======>.......................] - ETA: 1s - loss: 0.3533 - accuracy: 0.8498
Epoch 00044: saving model to checkpoints/weights.44.hdf5
 7616/27317 [=======>......................] - ETA: 1s - loss: 0.3520 - accuracy: 0.8518
Epoch 00044: saving model to checkpoints/weights.44.hdf5
 8640/27317 [========>.....................] - ETA: 1s - loss: 0.3515 - accuracy: 0.8527
Epoch 00044: saving model to checkpoints/weights.44.hdf5
 9632/27317 [=========>....................] - ETA: 1s - loss: 0.3520 - accuracy: 0.8537

KeyboardInterrupt: 

In [12]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

KeyError: 0